# Recall of cause metrics

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
SOCKSHOP_DATASET_ID = "9n6mf"

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

In [4]:
import sys
sys.path.append('../')

In [5]:
from meltria import loader
from eval import validation

In [6]:
metrics_files = !find "/datasets/argowf-chaos-{SOCKSHOP_DATASET_ID}/" -type f -name "*.json"
records = loader.load_dataset(metrics_files, target_metric_types={
        "containers": True,
        "services": True,
        "nodes": True,
        "middlewares": True,
    },
    num_datapoints=4*45, # 45min
)

In [7]:
well_injected_records = validation.find_records_detected_anomalies_of_sli(records, faulty_datapoints=4*5, sli_index=0)
well_injected_records = validation.find_records_detected_anomalies_of_cause_metrics(well_injected_records, faulty_datapoints=4*5)

In [8]:
from eval.tsdr import sweep_tsdr_for_recall

In [18]:
score_df = sweep_tsdr_for_recall(
    records=well_injected_records, 
    list_of_tsdr_options=[
        {
            "enable_unireducer": True,
            "enable_multireducer": True,
            # "step1_enable_smoother": False,
            # "step1_smoother_window_size": 4,
            "step1_method_name": "residual_integral",
            "step1_residual_integral_threshold": 20,
            "step1_residual_integral_change_start_point": False,
            "step1_residual_integral_change_start_point_n_sigma": 3,
            # "step2_enable_smoother": True,
            # "step2_smoother_window_size": 4,
            "step2_dbscan_min_pts": 1,
            "step2_dbscan_dist_type": "pearsonr",  # 'pearsonr' or 'sbd'
            "step2_dbscan_algorithm": "hdbscan",  # 'dbscan' or 'hdbscan'
            "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
            "step2_clustering_choice_method": "medoid",  # 'medoid' or 'maxsum'
        },
    ],
    use_manually_selected_metrics=[False],
)

In [19]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    cols = score_df.loc[:, score_df.columns.str.contains("n_|recall", regex=True)].columns.tolist()
    display(score_df.groupby(["metric_types/middlewares"])[cols].agg(["mean", "max", "min"]))
    display(score_df.groupby(["chaos_type", "metric_types/middlewares"])[cols].agg(["mean", "max", "min"]))
    display(score_df.groupby(["chaos_comp", "metric_types/middlewares"])[cols].agg(["mean", "max", "min"]))
    display(score_df.set_index(["chaos_type", "chaos_comp", "chaos_case_num", "metric_types/middlewares"]).sort_index())

/tmp/ipykernel_2803061/1822756972.py:3: FutureWarning: ['step2_dbscan_dist_type', 'step2_dbscan_algorithm'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  display(score_df.groupby(["metric_types/middlewares"])[cols].agg(["mean", "max", "min"]))


filtered/mand_recall                 \
                                         mean  max       min   
metric_types/middlewares                                       
False                                0.996396  1.0  0.800000   
True                                 0.982328  1.0  0.714286   

                         filtered/n_total_mand_metrics          \
                                                  mean max min   
metric_types/middlewares                                         
False                                         4.090090   5   3   
True                                          9.261261  28   3   

                         filtered/n_found_mand_metrics          \
                                                  mean max min   
metric_types/middlewares                                         
False                                         4.072072   5   3   
True                                          8.774775  26   3   

                         filtered/recall                      \
                                    mean       max       min   
metric_types/middlewares                                       
False                           0.729129  0.800000  0.444444   
True                            0.795137  0.925926  0.656250   

                         filtered/n_total_metrics          \
                                             mean max min   
metric_types/middlewares                                    
False                                    9.684685  16   5   
True                                    16.207207  32   5   

                         filtered/n_found_metrics          \
                                             mean max min   
metric_types/middlewares                                    
False                                    7.009009  12   4   
True                                    13.063063  26   4   

                         anomalous/mand_recall            \
                                          mean  max  min   
metric_types/middlewares                                   
False                                 0.964865  1.0  0.0   
True                                  0.947584  1.0  0.0   

                         anomalous/n_total_mand_metrics          \
                                                   mean max min   
metric_types/middlewares                                          
False                                          4.090090   5   3   
True                                           9.261261  28   3   

                         anomalous/n_found_mand_metrics          \
                                                   mean max min   
metric_types/middlewares                                          
False                                          3.945946   5   0   
True                                           8.486486  25   0   

                         anomalous/recall                 \
                                     mean       max  min   
metric_types/middlewares                                   
False                            0.696584  0.800000  0.0   
True                             0.752930  0.925926  0.0   

                         anomalous/n_total_metrics          \
                                              mean max min   
metric_types/middlewares                                     
False                                     9.684685  16   5   
True                                     16.207207  32   5   

                         anomalous/n_found_metrics          \
                                              mean max min   
metric_types/middlewares                                     
False                                     6.765766  12   0   
True                                     12.477477  25   0   

                         reduced/mand_recall                 \
                                        mean       max  min   
metric_types/middlewares                                      
False                   

/tmp/ipykernel_2803061/1822756972.py:4: FutureWarning: ['step2_dbscan_dist_type', 'step2_dbscan_algorithm'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  display(score_df.groupby(["chaos_type", "metric_types/middlewares"])[cols].agg(["mean", "max", "min"]))


filtered/mand_recall       \
                                                             mean  max   
chaos_type          metric_types/middlewares                             
pod-cpu-hog         False                                1.000000  1.0   
                    True                                 1.000000  1.0   
pod-memory-hog      False                                0.990476  1.0   
                    True                                 0.964286  1.0   
pod-network-latency False                                1.000000  1.0   
                    True                                 0.987526  1.0   

                                                        \
                                                   min   
chaos_type          metric_types/middlewares             
pod-cpu-hog         False                     1.000000   
                    True                      1.000000   
pod-memory-hog      False                     0.800000   
                    True                      0.714286   
pod-network-latency False                     1.000000   
                    True                      0.884615   

                                             filtered/n_total_mand_metrics  \
                                                                      mean   
chaos_type          metric_types/middlewares                                 
pod-cpu-hog         False                                         3.000000   
                    True                                          5.281250   
pod-memory-hog      False                                         5.000000   
                    True                                         11.404762   
pod-network-latency False                                         4.000000   
                    True                                         10.270270   

                                                      \
                                             max min   
chaos_type          metric_types/middlewares           
pod-cpu-hog         False                      3   3   
                    True                       8   3   
pod-memory-hog      False                      5   5   
                    True                      28   5   
pod-network-latency False                      4   4   
                    True                      26   4   

                                             filtered/n_found_mand_metrics  \
                                                                      mean   
chaos_type          metric_types/middlewares                                 
pod-cpu-hog         False                                         3.000000   
                    True                                          5.281250   
pod-memory-hog      False                                         4.952381   
                    True                                         10.404762   
pod-network-latency False                                         4.000000   
                    True                                          9.945946   

                                                     filtered/recall  \
                                             max min            mean   
chaos_type          metric_types/middlewares                           
pod-cpu-hog         False                      3   3        0.750000   
                    True                       8   3        0.777863   
pod-memory-hog      False                      5   4        0.650794   
                    True                      26   5        0.753671   
pod-network-latency False                      4   4        0.800000   
                    True                      25   4        0.857148   

                                                                  \
                                                   max       min   
chaos_type          metric_types/middlewares                       
pod-cpu-hog         False                     0.750000  0.750000   
                    Tru

/tmp/ipykernel_2803061/1822756972.py:5: FutureWarning: ['step2_dbscan_dist_type', 'step2_dbscan_algorithm'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  display(score_df.groupby(["chaos_comp", "metric_types/middlewares"])[cols].agg(["mean", "max", "min"]))


filtered/mand_recall                 \
                                                      mean  max       min   
chaos_comp   metric_types/middlewares                                       
carts        False                                1.000000  1.0  1.000000   
             True                                 0.854945  1.0  0.714286   
carts-db     False                                1.000000  1.0  1.000000   
             True                                 1.000000  1.0  1.000000   
catalogue    False                                1.000000  1.0  1.000000   
             True                                 1.000000  1.0  1.000000   
catalogue-db False                                1.000000  1.0  1.000000   
             True                                 1.000000  1.0  1.000000   
orders       False                                0.963636  1.0  0.800000   
             True                                 0.953546  1.0  0.892857   
orders-db    False                                1.000000  1.0  1.000000   
             True                                 1.000000  1.0  1.000000   
payment      False                                1.000000  1.0  1.000000   
             True                                 1.000000  1.0  1.000000   
user         False                                1.000000  1.0  1.000000   
             True                                 1.000000  1.0  1.000000   
user-db      False                                1.000000  1.0  1.000000   
             True                                 1.000000  1.0  1.000000   

                                      filtered/n_total_mand_metrics          \
                                                               mean max min   
chaos_comp   metric_types/middlewares                                         
carts        False                                         4.200000   5   3   
             True                                         22.800000  28   6   
carts-db     False                                         4.071429   5   3   
             True                                          9.785714  11   8   
catalogue    False                                         4.071429   5   3   
             True                                          4.071429   5   3   
catalogue-db False                                         4.000000   5   3   
             True                                          4.000000   5   3   
orders       False                                         4.000000   5   3   
             True                                         19.454545  28   6   
orders-db    False                                         4.181818   5   3   
             True                                          9.909091  11   8   
payment      False                                         4.300000   5   3   
             True                                          4.300000   5   3   
user         False                                         4.000000   5   3   
             True                                          4.000000   5   3   
user-db      False                                         4.071429   5   3   
             True                                          9.428571  10   8   

                                      filtered/n_found_mand_metrics          \
                                                               mean max min   
chaos_comp   metric_types/middlewares                                         
carts        False                                         4.200000   5   3   
             True                                         18.800000  25   6   
carts-db     False                                         4.071429   5   3   
             True                                          9.785714  11   8   
catalogue    False                                         4.071429   5   3   
             True                                          4.071429   5   3   
catalogue-db False                               

filtered/mand_recall  \
chaos_type          chaos_comp   chaos_case_num metric_types/middlewares                         
pod-cpu-hog         carts        2              False                                 1.000000   
                                                True                                  1.000000   
                                 4              False                                 1.000000   
                                                True                                  1.000000   
                    carts-db     0              False                                 1.000000   
                                                True                                  1.000000   
                                 1              False                                 1.000000   
                                                True                                  1.000000   
                                 2              False                                 1.000000   
                                                True                                  1.000000   
                                 4              False                                 1.000000   
                                                True                                  1.000000   
                    catalogue    0              False                                 1.000000   
                                                True                                  1.000000   
                                 1              False                                 1.000000   
                                                True                                  1.000000   
                                 2              False                                 1.000000   
                                                True                                  1.000000   
                                 4              False                                 1.000000   
                                                True                                  1.000000   
                    catalogue-db 1              False                                 1.000000   
                                                True                                  1.000000   
                                 2              False                                 1.000000   
                                                True                                  1.000000   
                                 3              False                                 1.000000   
                                                True                                  1.000000   
                                 4              False                                 1.000000   
                                                True                                  1.000000   
                    orders       1              False                                 1.000000   
                                                True                                  1.000000   
                                 2              False                                 1.000000   
                                                True                                  1.000000   
                                 3              False                                 1.000000   
                                                True                                  1.000000   
                                 4              False                                 1.000000   
                                                True                                  1.000000   
                    orders-db    0              False                                 1.000000   
                                                True                                  1.000000   
                                 1              False                                 1.000000   
                                                True    